### **SETUP & IMPORTS**

In [22]:
!pip install biopython==1.84

In [23]:
# Import required libraries and packages
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.SeqUtils import ProtParamData
import numpy as np
import pandas as pd

In [24]:
# Import files
from google.colab import files
upload=files.upload()

Saving neg_train.tsv to neg_train (1).tsv
Saving negative.fasta to negative (1).fasta
Saving pos_train.tsv to pos_train (1).tsv
Saving positive.fasta to positive (1).fasta


### **DATA PREPARATION**

### 1. Loading Training Data
Load the training dataset as **Pandas DataFrames**, separating positive and negative training sets, which will be used to train the classifier.

In [25]:
# Open training sets as DataFrames
pos_train=pd.read_csv('pos_train.tsv', sep='\t')
neg_train=pd.read_csv('neg_train.tsv', sep='\t')

### 2. Mapping FASTA sequences to DataFrames
Map each FASTA sequence to its corresponding entry in the DataFrame using the function `create_df`

In [26]:
# Map FASTA sequences to the DataFrames
def create_df(input_fasta, input_df):

    # Store sequences with their IDs
    fasta_dict = {}
    with open(input_fasta, 'r') as reader:
        seq_id = None
        seq_lines = []
        for line in reader:
            line = line.strip()
            if not line:
                continue    # Skip empty lines

            # Start of a new sequence
            if line.startswith('>'):
                if seq_id:
                    fasta_dict[seq_id] = ''.join(seq_lines)
                seq_id = line[1:].split()[0]
                seq_lines = []
            else:
                # Add sequence lines
                seq_lines.append(line)
        if seq_id:
            fasta_dict[seq_id] = ''.join(seq_lines)

    # Add the sequences to the DataFrame
    input_df['sequence'] = input_df['seq_id'].map(fasta_dict)
    return input_df


In [27]:
# Add the FASTA sequences to the positive and negative training DataFrames
pos_train = create_df('positive.fasta', pos_train)
neg_train = create_df('negative.fasta', neg_train)

### **DEFINING AMINO ACID PROPERTY SCALES**

We define 5 **amino acid scales** that capture key biochemical and structural features of signal peptides (SPs):

- hydropathicity
- charge
- polarity
- α-Helix propensity
- size / volume

Some of the residue property scales used here are derived from:
- **ProtScale (ExPASy):** A collection of amino acid property scales (e.g., hydrophobicity, flexibility, charge, polarity).  
- **AAindex:** A curated database of numerical indices representing physical and chemical properties of amino acids.

These resources are used to build **custom property dictionaries**, ensuring that the computed features reflect biologically validated residue properties.

Each protein sequence is therefore represented by a fixed-length vector of numerical descriptors,
making it suitable for **Support Vector Machine (SVM)** input.


In [28]:
# Define amino acid scales
my_scale_hydropathicity = {'A': 1.8,'R': -4.5,'N': -3.5,'D': -3.5,'C': 2.5,'Q': -3.5,'E': -3.5,
                           'G': -0.4,'H': -3.2,'I': 4.5,'L': 3.8,'K': -3.9,'M': 1.9,'F': 2.8,
                           'P': -1.6,'S': -0.8,'T': -0.7,'W': -0.9,'Y': -1.3,'V': 4.2}
my_scale_charge = {'A':0,'R':1,'N':0,'D':0,'C':0,'Q':0,'E':0,'G':0,'H':1,'I':0,'L':0,'K':1,
                   'M':0,'F':0,'P':0,'S':0,'T':0,'W':0,'Y':0,'V':0}
my_scale_alpha_helix = {'A':1.42,'R':0.98,'N':0.67,'D':1.01,'C':0.70,'Q':1.11,'E':1.51,'G':0.57,
                        'H':1.00,'I':1.08,'L':1.21,'K':1.16,'M':1.45,'F':1.13,'P':0.57,'S':0.77,
                        'T':0.83,'W':1.08,'Y':0.69,'V':1.06}
my_scale_transmembrane = {'A':0.38,'R':-2.57,'N':-1.62,'D':-3.27,'C':-0.30,'Q':-1.84,'E':-2.90,
                          'G':-0.19,'H':-1.44,'I':1.97,'L':1.82,'K':-2.03,'M':1.40,'F':1.35,
                          'P':-1.44,'S':-0.53,'T':-0.32,'W':1.07,'Y':0.26,'V':1.46}
my_scale_size = {'A':11.50,'R':14.28,'N':12.82,'D':11.68,'C':13.46,'Q':14.45,'E':13.57,'G':3.40,
                 'H':13.69,'I':21.40,'L':21.40,'K':15.71,'M':16.25,'F':19.80,'P':12.41,'S':9.47,
                 'T':15.77,'W':25.62,'Y':18.03,'V':21.57}


### **DEFINING THE AMINO ACID ALPHABET**

The standard amino acid alphabet serves as a reference for encoding each sequence, helping ensure consistency when mapping residues to numerical values.

In [29]:
# Define amino acid alphabet
alphabet = list('GAVLIMFWYSTCNQHDEKR')

### **SEQUENCE PADDING**
To ensure all sequences have equal length, we apply padding.  
Shorter sequences are extended with a neutral placeholder (often 'X' or 0), allowing them to be represented as fixed-size numerical vectors for machine learning.

In [30]:
# Padding the sequence
def padding(sequence, wd=5):
    d = int(wd / 2)
    return "X" * d + sequence + "X" * d

### **EXTRACTING BIOPHYSICAL FEATURES FROM PROTEIN SEQUENCES**

This function, `extract_features()`, converts a single amino acid sequence into a numerical feature vector
based on multiple biophysical property scales.

1. **Padding:** The sequence is first padded to ensure a uniform length across all samples.  
2. **Sliding-window analysis:** The function applies a sliding window (size = 5 residues) to compute local profiles
   for several residue property scales:
   - Hydropathicity  
   - Charge  
   - α-Helix Propensity  
   - Transmembrane Tendency  
   - Size / Volume  
3. **Feature summarization:** For each property, both **maximum** and **mean** values are calculated,
   capturing local and global characteristics of the sequence.

The output is a compact numerical vector describing the sequence’s physicochemical composition,
which will later serve as input features for the **SVM classifier**.


In [31]:
# Extract biophysical features for one sequence
def extract_features(sequence):
    seq_pad = padding(sequence)
    analysed = ProteinAnalysis(seq_pad)

    # Compute sliding-window property profiles
    hydro = analysed.protein_scale(my_scale_hydropathicity, 5)
    charge = analysed.protein_scale(my_scale_charge, 5)
    alpha = analysed.protein_scale(my_scale_alpha_helix, 5)
    trans = analysed.protein_scale(my_scale_transmembrane, 5)
    size = analysed.protein_scale(my_scale_size, 5)

    # Return max and mean for each scale
    feats = [
        np.max(hydro), np.mean(hydro),
        np.max(charge), np.mean(charge),
        np.max(alpha), np.mean(alpha),
        np.max(trans), np.mean(trans),
        np.max(size), np.mean(size)
    ]
    return feats

### **COMPUTE AMINO ACIDS COMPOSITION**
This function calculates the amino acid composition for each sequence over the first 22 residues.

In [32]:
# Compute amino acid composition for each sequence over the first 22 residues
def aa_composition(sequence, alphabet, length=22):
    if pd.isna(sequence):
        return np.zeros(len(alphabet))
    seq = sequence[:length]
    comp = np.zeros(len(alphabet))
    for i, aa in enumerate(alphabet):
        comp[i] = seq.count(aa) / len(seq)
    return comp

### **EXTRACT ALL NUMERICAL AND COMPOSITION FEATURES**

`extract_all()` extracts all relevant features from a dataset of sequences. It includes:

- **Numerical features**: such as hydrophobicity, charge, alpha and transmembrane propensities, and size, with both maximum and mean values.
- **Amino acid composition**: the fraction of each amino acid in the first 22 residues of the sequence.

All features are combined into a single DataFrame along with sequence IDs for further analysis.

In [33]:
# Extract all numerical and composition features for each sequence
def extract_all(df):

    # Numerical features
    features = df['sequence'].apply(extract_features)
    arr = np.vstack(features)
    cols = [
        'hydro_max','hydro_mean',
        'charge_max','charge_mean',
        'alpha_max','alpha_mean',
        'trans_max','trans_mean',
        'size_max','size_mean'
    ]

    # Amino acid composition
    comp_list = []
    for seq in df['sequence']:
        comp_list.append(aa_composition(seq, alphabet))
    comp_arr = np.vstack(comp_list)
    comp_cols = [f'comp_{a}' for a in alphabet]

    # Merge all features in a single DataFrame
    all_arr = np.hstack([arr, comp_arr])
    all_cols = cols + comp_cols
    feat_df = pd.DataFrame(all_arr, columns=all_cols)
    feat_df['seq_id'] = df['seq_id']

    return feat_df

In [34]:
# Extract features for positive and negative training sets
pos_features = extract_all(pos_train)
neg_features = extract_all(neg_train)

# Assign class labels
pos_features['class'] = 1
neg_features['class'] = 0

# Combine positive and negative features into one dataset
dataset = pd.concat([pos_features, neg_features], ignore_index=True)


Streaming output truncated to the last 5000 lines.


### **RETRIEVE SEQUENCES BY ID**

- **Function `get_sequence_by_id`**:  
  - Takes a sequence ID as input.  
  - Searches for the ID in both positive (`pos_train`) and negative (`neg_train`) training sets.  
  - Returns the corresponding sequence if found, or `NaN` if the ID is missing.  

- **Mapping sequences to the dataset**:  
  - The function is applied to all sequence IDs in the dataset.  
  - A new column `sequence` is added containing the actual amino acid sequences for each ID.


In [35]:
# Find the sequence corresponding to a given ID
def get_sequence_by_id(id_):
    if id_ in pos_train['seq_id'].values:
        return pos_train.loc[pos_train['seq_id'] == id_, 'sequence'].values[0]
    elif id_ in neg_train['seq_id'].values:
        return neg_train.loc[neg_train['seq_id'] == id_, 'sequence'].values[0]
    else:
        return np.nan  # handle missing case

# Map sequences for all IDs in the dataset
dataset['sequence'] = dataset['seq_id'].map(get_sequence_by_id)

In [36]:
dataset['sequence'] = dataset['seq_id'].map(get_sequence_by_id)

# FINAL FEATURES AND LABELS
X = dataset.drop(columns=['seq_id', 'class', 'sequence'])
y = dataset['class']

print("Feature matrix shape:", X.shape)
print("Feature names:", X.columns.tolist())

# SAVE RESULTS
output_file = 'ML_features.tsv'
dataset.to_csv(output_file, sep='\t', index=False)

print(f"\nFeature extraction completed successfully.")
print(f"File saved as: {output_file}")
print(f"Total samples: {dataset.shape[0]}, Total features: {X.shape[1]}")

Feature matrix shape: (8021, 29)
Feature names: ['hydro_max', 'hydro_mean', 'charge_max', 'charge_mean', 'alpha_max', 'alpha_mean', 'trans_max', 'trans_mean', 'size_max', 'size_mean', 'comp_G', 'comp_A', 'comp_V', 'comp_L', 'comp_I', 'comp_M', 'comp_F', 'comp_W', 'comp_Y', 'comp_S', 'comp_T', 'comp_C', 'comp_N', 'comp_Q', 'comp_H', 'comp_D', 'comp_E', 'comp_K', 'comp_R']

Feature extraction completed successfully.
File saved as: ML_features.tsv
Total samples: 8021, Total features: 29
